In [1]:
# Importing necessary packages
import pandas as pd
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import os
import imutils
import numpy as np
%matplotlib inline

In [2]:
# Checking if GPU available
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

IndexError: list index out of range

In [4]:
# Function for getting text from images
def get_text(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray_t = cv2.threshold(gray, 0, 255,
                             cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    gray_b = cv2.medianBlur(gray, 3)

    # Writing the grayscale image to disk as a temporary file so we can
    # apply OCR to it
    filename = "Gray/{}.png".format(os.getpid())
    cv2.imwrite(filename, gray_t)

    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text_t = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    
    filename = "Gray/{}.png".format(os.getpid())
    cv2.imwrite(filename, gray_b)

    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text_b = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    
    text = text_t + ' ' + text_b
    text = ' '.join(text.split())
    return text

In [5]:
# Loading saved model (created in lstm.ipynb and saved in drive)
model = load_model('lstm_model.h5')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 25, 150)           4500000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 4,781,202
Trainable params: 4,781,202
Non-trainable params: 0
_________________________________________________________________


In [6]:
nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /home/err_pv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Preprocess text and get model prediction
def preprocess(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)

    max_fatures = 30000
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(text)
    text = tokenizer.texts_to_sequences(text)
#     print(text, len(text))
    if len(text)==0:
        return 'Random'
    text = pad_sequences(text)
    text = text[:25]
#     print(text)
    result = model.predict(text.reshape(1,len(text)),batch_size=1,verbose = 2)[0]
    
    if np.argmax(result)==0:
        return 'Negative'
    return 'Positive'

    

In [8]:
file = pd.read_csv('Test.csv')

# Converting category column to string
file['Category'] = file['Category'].astype(str)
file.head()

,Filename,Category
0,Test1001.jpg,nan
1,Test1012.jpg,nan
2,Test1022.jpg,nan
3,Test1071.jpg,nan
4,Test1122.jpg,nan


In [18]:
# Making predictions
j = 0
for i, row in file.iterrows():
    j+=1
    if j<118:
        continue
#     print(j)
    img_name = row[0]
    img_path = '../../Data Files/Dataset/'+img_name
    image = cv2.imread(img_path)
    quote = get_text(image)
    if len(quote)<5:
        file.at[i, 'Category'] = 'Random'
    else:
        ans = preprocess(quote)
        file.at[i, 'Category'] = ans
    print(j, ans)
#     if j==2:
#         break
        
file.head()        

118 Negative
119 Random
1/1 - 0s
120 Positive
121 Positive
1/1 - 0s
122 Positive
1/1 - 0s
123 Positive
124 Positive
1/1 - 0s
125 Negative
1/1 - 0s
126 Positive
127 Positive
1/1 - 0s
128 Positive
129 Positive
130 Positive
1/1 - 0s
131 Negative
1/1 - 0s
132 Positive
1/1 - 0s
133 Negative
1/1 - 0s
134 Positive
1/1 - 0s
135 Negative
1/1 - 0s
136 Negative
1/1 - 0s
137 Negative
1/1 - 0s
138 Positive
139 Positive
1/1 - 0s
140 Positive
1/1 - 0s
141 Negative
1/1 - 0s
142 Positive
1/1 - 0s
143 Positive
1/1 - 0s
144 Positive
1/1 - 0s
145 Negative
1/1 - 0s
146 Positive
1/1 - 0s
147 Negative
1/1 - 0s
148 Positive
1/1 - 0s
149 Positive
1/1 - 0s
150 Positive
1/1 - 0s
151 Negative
1/1 - 0s
152 Negative
1/1 - 0s
153 Positive
1/1 - 0s
154 Positive
1/1 - 0s
155 Positive
1/1 - 0s
156 Positive
1/1 - 0s
157 Negative
158 Negative
1/1 - 0s
159 Negative
160 Negative
1/1 - 0s
161 Positive
1/1 - 0s
162 Negative
1/1 - 0s
163 Negative
1/1 - 0s
164 Negative
165 Negative
166 Negative
167 Negative
168 Negative
1/1 - 

,Filename,Category
0,Test1001.jpg,Positive
1,Test1012.jpg,Positive
2,Test1022.jpg,Positive
3,Test1071.jpg,Negative
4,Test1122.jpg,Negative


In [19]:
print(file['Category'].value_counts())
file.head()

Random      104
Positive     84
Negative     51
Name: Category, dtype: int64


,Filename,Category
0,Test1001.jpg,Positive
1,Test1012.jpg,Positive
2,Test1022.jpg,Positive
3,Test1071.jpg,Negative
4,Test1122.jpg,Negative


In [20]:
file.to_csv('Test.csv')

In [12]:
# Performance
# Score
# 46.33494